In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [2]:
fake = faker.Faker(locale='ru')

In [3]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip()

In [4]:
HH_API_URL = 'http://api.hh.ru/'
HH_TOKEN_URL = "https://hh.ru/oauth/token"

In [5]:
CLIENT_ID = "N9N75GOTCRKJAIKK8TVM5UNPI38GC0B39BT0K2Q1AUM3P35LMCPJHQLDC952PH2A"
CLIENT_SECRET = "O2VFGTA91FPTUHDLSUVBOVU3JETH3RE2VBUERKRGP8SK0GGCQ5LCJSIKA5FGAIRB"

In [6]:
token_result = requests.post(HH_TOKEN_URL, data={
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET
})

In [7]:
token = token_result.json()['access_token']
token

'HSPF996J781VSGN0SR2PQ1TOSOO485H5TGS4HVV6N1D7PB9GEI0E46HRM70780VF'

In [8]:
hh_headers = {
    'Authorization': f'Bearer {token}'
}

In [9]:
hh_dictionaries = requests.get(HH_API_URL + 'dictionaries').json()
hh_dictionaries['vacancy_search_order'], hh_dictionaries['vacancy_search_order']

([{'id': 'publication_time', 'name': 'по дате'},
  {'id': 'salary_desc', 'name': 'по убыванию дохода'},
  {'id': 'salary_asc', 'name': 'по возрастанию дохода'},
  {'id': 'relevance', 'name': 'по соответствию'},
  {'id': 'distance', 'name': 'по удалённости'}],
 [{'id': 'publication_time', 'name': 'по дате'},
  {'id': 'salary_desc', 'name': 'по убыванию дохода'},
  {'id': 'salary_asc', 'name': 'по возрастанию дохода'},
  {'id': 'relevance', 'name': 'по соответствию'},
  {'id': 'distance', 'name': 'по удалённости'}])

In [12]:
query_params = {
    'text': 'МТС',
    'per_page': 50, # max 100
    'page': 0,
    'specialization': 1, # IT
    'order_by': 'publication_time',
    'search_field': 'company_name'
}

In [13]:
test_request = requests.get(HH_API_URL + 'vacancies', headers=hh_headers, params=query_params).json()
test_request['pages'], test_request['found']

(14, 678)

In [19]:
job_dicts = []
for page in tqdm(range(test_request['pages'])):
    query_params.update({'page': page})
    items = requests.get(HH_API_URL + 'vacancies', headers=hh_headers, params=query_params).json()['items']

    for item in items:
        job_info = requests.get(HH_API_URL + f'vacancies/{item["id"]}', headers=hh_headers).json()
        job_dict = {
            'title': clear_string(job_info['name']),
            'internal_id' : job_info['id'],
            'url': job_info['alternate_url'],
            'description': clear_string(job_info['description']),
            'responsibilities': None,
            'qualifications': None,
            'company': 'MTS',
            'location': job_info['address']['raw'] if job_info['address'] else None,
            'department': job_info['department']['name'] if job_info['department'] else None,
            'publish_date': parser.parse(job_info['created_at'])
        }
        job_dicts.append(job_dict)

    time.sleep(0.2) # 200 ms sleep
len(job_dicts)

100%|██████████| 14/14 [02:53<00:00, 12.37s/it]


678

In [20]:
snapshot = pd.DataFrame(filter(lambda x: x is not None, job_dicts))
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,company,location,department,publish_date
174,Ведущий бизнес-аналитик в центр развития фикси...,67698984,https://hh.ru/vacancy/67698984,МТС – это мультисервисная цифровая экосистема....,None,None,MTS,None,«МТС»,2022-08-08 09:08:53+03:00
299,QA Engineer,68393548,https://hh.ru/vacancy/68393548,Обязанности: Организация процесса тестирования...,None,None,MTS,None,"«МТС», IT",2022-08-02 13:25:01+03:00
606,Go developer middle (Ocean),54642361,https://hh.ru/vacancy/54642361,"MTS Digital – дочерняя компания ПАО МТС, задач...",None,None,MTS,None,"«МТС», IT",2022-07-18 09:36:39+03:00
601,Старший инженер DevOps,67039779,https://hh.ru/vacancy/67039779,В Стрим МТС Инвестиции ищем Старшего инженера ...,None,None,MTS,"Москва, проспект Андропова, 18к1",МТС Банк. IT,2022-07-18 10:52:12+03:00
583,Ведущий инженер опорной и ЦОД сетей,66571106,https://hh.ru/vacancy/66571106,МТС – это мультисервисная цифровая экосистема....,None,None,MTS,"Новосибирск, улица Богдана Хмельницкого, 90/1",«МТС»,2022-07-19 04:47:40+03:00


In [21]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype                                
---  ------            --------------  -----                                
 0   title             678 non-null    object                               
 1   internal_id       678 non-null    object                               
 2   url               678 non-null    object                               
 3   description       678 non-null    object                               
 4   responsibilities  0 non-null      object                               
 5   qualifications    0 non-null      object                               
 6   company           678 non-null    object                               
 7   location          342 non-null    object                               
 8   department        671 non-null    object                               
 9   publish_date      678 non-null    datetime6

In [22]:
snapshot['publish_date'] = snapshot['publish_date'].apply(lambda x: x.replace(tzinfo=None))
snapshot['publish_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 678 entries, 0 to 677
Series name: publish_date
Non-Null Count  Dtype         
--------------  -----         
678 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 5.4 KB


In [23]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'10-08-2022'

In [24]:
snapshot.to_csv(f'../data/mts/{current_date}.csv')
snapshot.to_excel(f'../data/mts/{current_date}.xlsx', engine='xlsxwriter')